# Análise e Segmentação de Clientes - Priceless Bank
## Mastercard Challenge 2025

Esta análise tem como objetivo segmentar os clientes do Priceless Bank utilizando dados demográficos como:
- **Idade** (calculada a partir da data de nascimento)
- **Renda Anual**
- **Localização** (Cidade e Estado)
- **Número de Cartões**
- **Conta Adicional**

O objetivo é identificar grupos distintos de clientes para personalizar estratégias de marketing e produtos.

## 1. Importar Bibliotecas Necessárias

In [ ]:
# Bibliotecas para manipulação de dados
import pandas as pd
import numpy as np
from datetime import datetime

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Bibliotecas para machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Configurações de visualização
sns.set_palette('husl')

# Configurar warnings
import warnings
warnings.filterwarnings('ignore')

print('✓ Bibliotecas importadas com sucesso!')

## 2. Carregar e Explorar a Base de Dados

In [ ]:
# Carregar base de clientes
df_clientes = pd.read_csv('Base_clientes.csv')

print(f"📊 Total de clientes: {len(df_clientes):,}")
print(f"📋 Colunas disponíveis: {list(df_clientes.columns)}")
print("\n" + "="*80)
df_clientes.head(10)

In [ ]:
# Informações sobre o dataset
print("📈 Informações do Dataset:")
print("="*80)
df_clientes.info()
print("\n" + "="*80)
print("\n📊 Estatísticas Descritivas:")
df_clientes.describe()

In [ ]:
# Verificar valores ausentes
print("❓ Valores Ausentes por Coluna:")
print("="*80)
missing_data = pd.DataFrame({
    'Total Missing': df_clientes.isnull().sum(),
    'Percentual (%)': (df_clientes.isnull().sum() / len(df_clientes) * 100).round(2)
})
missing_data = missing_data[missing_data['Total Missing'] > 0].sort_values('Total Missing', ascending=False)
print(missing_data)

## 3. Preparação e Engenharia de Features

In [ ]:
# Criar cópia do dataset para manipulação
df = df_clientes.copy()

# Converter Data_Nascimento para datetime
df['Data_Nascimento'] = pd.to_datetime(df['Data_Nascimento'], format='%d/%m/%Y', errors='coerce')
df['Data_Criacao_Conta'] = pd.to_datetime(df['Data_Criacao_Conta'], errors='coerce')

# Calcular idade atual
data_referencia = datetime(2025, 10, 3)
df['Idade'] = ((data_referencia - df['Data_Nascimento']).dt.days / 365.25).round(0)

# Calcular tempo como cliente (em anos)
df['Tempo_Cliente_Anos'] = ((data_referencia - df['Data_Criacao_Conta']).dt.days / 365.25).round(2)

# Criar faixas etárias
df['Faixa_Etaria'] = pd.cut(df['Idade'], 
                             bins=[0, 25, 35, 45, 55, 65, 100],
                             labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+'])

# Criar faixas de renda
df['Faixa_Renda'] = pd.cut(df['Renda_Anual'], 
                            bins=[0, 30000, 50000, 80000, 120000, np.inf],
                            labels=['Até 30k', '30k-50k', '50k-80k', '80k-120k', 'Acima 120k'])

# Converter Possui_Conta_Adicional para binário
df['Possui_Conta_Adicional_Bin'] = df['Possui_Conta_Adicional'].map({'Sim': 1, 'Não': 0})

print("✓ Features criadas com sucesso!")
print(f"\n📊 Dataset atualizado: {df.shape[0]} linhas x {df.shape[1]} colunas")
df.head()

## 4. Análise Demográfica - Distribuição por Idade

In [ ]:
# Visualização interativa da distribuição de idade
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição de Idade', 'Box Plot - Idade', 
                    'Distribuição por Faixa Etária', 'Idade Média por Estado'),
    specs=[[{'type': 'histogram'}, {'type': 'box'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Histograma de idade
fig.add_trace(
    go.Histogram(x=df['Idade'].dropna(), nbinsx=30, name='Idade',
                marker_color='rgb(55, 83, 109)'),
    row=1, col=1
)

# Box plot de idade
fig.add_trace(
    go.Box(y=df['Idade'].dropna(), name='Idade', marker_color='rgb(26, 118, 255)'),
    row=1, col=2
)

# Distribuição por faixa etária
faixa_etaria_count = df['Faixa_Etaria'].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=faixa_etaria_count.index, y=faixa_etaria_count.values,
          marker_color='rgb(50, 171, 96)', name='Faixa Etária'),
    row=2, col=1
)

# Idade média por estado
idade_por_estado = df.groupby('Estado')['Idade'].mean().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=idade_por_estado.index, y=idade_por_estado.values,
          marker_color='rgb(219, 64, 82)', name='Idade Média'),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=False, title_text="📊 Análise Completa - Idade dos Clientes")
fig.show()

# Estatísticas de idade
print("\n📈 Estatísticas de Idade:")
print("="*80)
print(f"Idade Média: {df['Idade'].mean():.1f} anos")
print(f"Idade Mediana: {df['Idade'].median():.1f} anos")
print(f"Idade Mínima: {df['Idade'].min():.0f} anos")
print(f"Idade Máxima: {df['Idade'].max():.0f} anos")
print(f"Desvio Padrão: {df['Idade'].std():.1f} anos")

## 5. Análise Demográfica - Distribuição por Renda

In [ ]:
# Análise de renda
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição de Renda Anual', 'Box Plot - Renda', 
                    'Distribuição por Faixa de Renda', 'Renda Média por Estado'),
    specs=[[{'type': 'histogram'}, {'type': 'box'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Histograma de renda
fig.add_trace(
    go.Histogram(x=df['Renda_Anual'].dropna(), nbinsx=40, name='Renda',
                marker_color='rgb(55, 128, 191)'),
    row=1, col=1
)

# Box plot de renda
fig.add_trace(
    go.Box(y=df['Renda_Anual'].dropna(), name='Renda', marker_color='rgb(128, 0, 128)'),
    row=1, col=2
)

# Distribuição por faixa de renda
faixa_renda_count = df['Faixa_Renda'].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=faixa_renda_count.index, y=faixa_renda_count.values,
          marker_color='rgb(255, 144, 14)', name='Faixa Renda'),
    row=2, col=1
)

# Renda média por estado
renda_por_estado = df.groupby('Estado')['Renda_Anual'].mean().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=renda_por_estado.index, y=renda_por_estado.values,
          marker_color='rgb(44, 160, 101)', name='Renda Média'),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=False, title_text="💰 Análise Completa - Renda dos Clientes")
fig.show()

# Estatísticas de renda
print("\n💵 Estatísticas de Renda Anual:")
print("="*80)
print(f"Renda Média: R$ {df['Renda_Anual'].mean():,.2f}")
print(f"Renda Mediana: R$ {df['Renda_Anual'].median():,.2f}")
print(f"Renda Mínima: R$ {df['Renda_Anual'].min():,.2f}")
print(f"Renda Máxima: R$ {df['Renda_Anual'].max():,.2f}")
print(f"Desvio Padrão: R$ {df['Renda_Anual'].std():,.2f}")

## 6. Análise Demográfica - Distribuição por Localização

In [ ]:
# Análise geográfica
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Top 10 Cidades', 'Distribuição por Estado', 
                    'Clientes por Estado (Pizza)', 'Renda Média vs Clientes por Estado'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'pie'}, {'type': 'scatter'}]]
)

# Top 10 cidades
top_cidades = df['Cidade'].value_counts().head(10)
fig.add_trace(
    go.Bar(y=top_cidades.index, x=top_cidades.values, orientation='h',
          marker_color='rgb(158, 202, 225)', name='Cidades'),
    row=1, col=1
)

# Distribuição por estado
estado_count = df['Estado'].value_counts()
fig.add_trace(
    go.Bar(x=estado_count.index, y=estado_count.values,
          marker_color='rgb(94, 204, 243)', name='Estados'),
    row=1, col=2
)

# Pizza - Estados
fig.add_trace(
    go.Pie(labels=estado_count.index, values=estado_count.values),
    row=2, col=1
)

# Scatter - Renda vs Clientes
estado_stats = df.groupby('Estado').agg({
    'Cliente_ID': 'count',
    'Renda_Anual': 'mean'
}).reset_index()
estado_stats.columns = ['Estado', 'Num_Clientes', 'Renda_Media']

fig.add_trace(
    go.Scatter(x=estado_stats['Num_Clientes'], y=estado_stats['Renda_Media'],
              mode='markers+text', text=estado_stats['Estado'],
              textposition='top center',
              marker=dict(size=15, color='rgb(255, 127, 14)'),
              name='Estados'),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False, title_text="🗺️ Análise Geográfica dos Clientes")
fig.show()

print("\n🌎 Estatísticas Geográficas:")
print("="*80)
print(f"Total de Estados: {df['Estado'].nunique()}")
print(f"Total de Cidades: {df['Cidade'].nunique()}")
print(f"\nTop 5 Estados por número de clientes:")
print(estado_count.head())

## 7. Análise Demográfica - Outros Dados Relevantes

In [ ]:
# Análise de outros dados relevantes
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição de Número de Cartões', 'Possui Conta Adicional',
                    'Tempo Médio como Cliente', 'Cartões vs Renda'),
    specs=[[{'type': 'bar'}, {'type': 'pie'}],
           [{'type': 'histogram'}, {'type': 'box'}]]
)

# Número de cartões
cartoes_count = df['Numero_Cartoes'].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=cartoes_count.index, y=cartoes_count.values,
          marker_color='rgb(99, 110, 250)', name='Cartões'),
    row=1, col=1
)

# Conta adicional
conta_adicional = df['Possui_Conta_Adicional'].value_counts()
fig.add_trace(
    go.Pie(labels=conta_adicional.index, values=conta_adicional.values,
          marker_colors=['rgb(239, 85, 59)', 'rgb(99, 110, 250)']),
    row=1, col=2
)

# Tempo como cliente
fig.add_trace(
    go.Histogram(x=df['Tempo_Cliente_Anos'].dropna(), nbinsx=20,
                marker_color='rgb(0, 204, 150)', name='Tempo'),
    row=2, col=1
)

# Box plot - Cartões vs Renda
for i in sorted(df['Numero_Cartoes'].unique()):
    fig.add_trace(
        go.Box(y=df[df['Numero_Cartoes']==i]['Renda_Anual'], name=f'{i} cartões'),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="📊 Análise de Outros Dados Demográficos")
fig.show()

print("\n📋 Estatísticas Adicionais:")
print("="*80)
print(f"Média de Cartões por Cliente: {df['Numero_Cartoes'].mean():.2f}")
print(f"% Clientes com Conta Adicional: {(df['Possui_Conta_Adicional_Bin'].mean()*100):.1f}%")
print(f"Tempo Médio como Cliente: {df['Tempo_Cliente_Anos'].mean():.2f} anos")

## 8. Análise de Correlação entre Variáveis

In [ ]:
# Matriz de correlação
correlation_vars = ['Idade', 'Renda_Anual', 'Numero_Cartoes', 
                    'Possui_Conta_Adicional_Bin', 'Tempo_Cliente_Anos']
corr_matrix = df[correlation_vars].corr()

# Heatmap de correlação
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values.round(2),
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Correlação")
))

fig.update_layout(
    title='🔗 Matriz de Correlação entre Variáveis',
    height=600,
    xaxis_title='',
    yaxis_title=''
)
fig.show()

print("\n🔍 Principais Correlações:")
print("="*80)
# Encontrar correlações mais fortes (excluindo diagonal)
corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_pairs.append((
            corr_matrix.columns[i],
            corr_matrix.columns[j],
            corr_matrix.iloc[i, j]
        ))
corr_pairs_sorted = sorted(corr_pairs, key=lambda x: abs(x[2]), reverse=True)
for var1, var2, corr in corr_pairs_sorted[:5]:
    print(f"{var1} <-> {var2}: {corr:.3f}")

## 9. Segmentação de Clientes - K-Means Clustering

In [ ]:
# Preparar dados para clustering
# Selecionar apenas clientes com dados completos
df_cluster = df[['Idade', 'Renda_Anual', 'Numero_Cartoes', 
                 'Possui_Conta_Adicional_Bin', 'Tempo_Cliente_Anos']].dropna()

print(f"📊 Clientes com dados completos para clustering: {len(df_cluster):,}")

# Padronizar os dados
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Método do cotovelo para determinar número ótimo de clusters
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertias.append(kmeans.inertia_)

# Plotar método do cotovelo
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(K_range),
    y=inertias,
    mode='lines+markers',
    marker=dict(size=10, color='rgb(55, 83, 109)'),
    line=dict(color='rgb(55, 83, 109)', width=2)
))

fig.update_layout(
    title='📈 Método do Cotovelo - Determinação do Número Ótimo de Clusters',
    xaxis_title='Número de Clusters',
    yaxis_title='Inércia (Within-Cluster Sum of Squares)',
    height=500
)
fig.show()

print("\n💡 Análise: O 'cotovelo' do gráfico sugere o número ótimo de clusters.")
print("Vamos utilizar 5 clusters para segmentação detalhada.")

In [ ]:
# Aplicar K-Means com 5 clusters
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df_cluster['Segmento'] = kmeans.fit_predict(df_scaled)

# Adicionar segmentos ao dataframe original
df_with_segments = df.copy()
df_with_segments.loc[df_cluster.index, 'Segmento'] = df_cluster['Segmento']

# Nomear os segmentos de forma descritiva
segment_names = {
    0: 'Segmento 0',
    1: 'Segmento 1',
    2: 'Segmento 2',
    3: 'Segmento 3',
    4: 'Segmento 4'
}

print(f"✓ Segmentação concluída! {n_clusters} segmentos identificados.")
print(f"\n📊 Distribuição de Clientes por Segmento:")
print("="*80)
segment_dist = df_cluster['Segmento'].value_counts().sort_index()
for seg, count in segment_dist.items():
    pct = (count / len(df_cluster)) * 100
    print(f"Segmento {seg}: {count:,} clientes ({pct:.1f}%)")

## 10. Visualização de Segmentos Identificados

In [ ]:
# PCA para visualização 2D
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)
df_cluster['PCA1'] = df_pca[:, 0]
df_cluster['PCA2'] = df_pca[:, 1]

# Scatter plot 2D com PCA
fig = px.scatter(
    df_cluster,
    x='PCA1',
    y='PCA2',
    color='Segmento',
    title='🎯 Visualização dos Segmentos (PCA 2D)',
    labels={'PCA1': f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variância)',
            'PCA2': f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variância)'},
    color_continuous_scale='Viridis',
    height=600
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.show()

print(f"\n📊 Variância explicada pelos 2 componentes principais: {sum(pca.explained_variance_ratio_):.1%}")

In [ ]:
# Visualização 3D
fig = px.scatter_3d(
    df_cluster,
    x='Idade',
    y='Renda_Anual',
    z='Numero_Cartoes',
    color='Segmento',
    title='🎯 Visualização 3D dos Segmentos (Idade x Renda x Cartões)',
    color_continuous_scale='Plasma',
    height=700,
    opacity=0.7
)
fig.update_traces(marker=dict(size=5))
fig.show()

In [ ]:
# Pair plot por segmento (usando subset para performance)
import plotly.figure_factory as ff

# Criar scatter matrix
fig = px.scatter_matrix(
    df_cluster.sample(min(1000, len(df_cluster))),  # Sample para performance
    dimensions=['Idade', 'Renda_Anual', 'Numero_Cartoes', 'Tempo_Cliente_Anos'],
    color='Segmento',
    title='📊 Matriz de Dispersão - Comparação entre Variáveis por Segmento',
    height=900,
    color_continuous_scale='Turbo'
)
fig.update_traces(diagonal_visible=False, marker=dict(size=4, opacity=0.6))
fig.show()

## 11. Perfil Detalhado de Cada Segmento

In [ ]:
# Análise detalhada por segmento
segment_profile = df_cluster.groupby('Segmento').agg({
    'Idade': ['mean', 'std'],
    'Renda_Anual': ['mean', 'std'],
    'Numero_Cartoes': ['mean', 'mode'],
    'Possui_Conta_Adicional_Bin': 'mean',
    'Tempo_Cliente_Anos': 'mean'
}).round(2)

# Flatten multi-index columns
segment_profile.columns = ['_'.join(col).strip() for col in segment_profile.columns.values]
segment_profile.columns = ['Idade_Média', 'Idade_Desvio', 'Renda_Média', 'Renda_Desvio',
                           'Cartões_Média', 'Cartões_Moda', 'Pct_Conta_Adicional', 'Tempo_Médio_Anos']

# Adicionar contagem de clientes
segment_profile['Num_Clientes'] = df_cluster['Segmento'].value_counts().sort_index()
segment_profile['Pct_Total'] = (segment_profile['Num_Clientes'] / len(df_cluster) * 100).round(1)

print("\n📋 PERFIL DETALHADO DOS SEGMENTOS")
print("="*120)
display(segment_profile.style.background_gradient(cmap='YlOrRd', subset=['Renda_Média', 'Idade_Média']))

In [ ]:
# Criar descrições textuais para cada segmento
print("\n🎯 CARACTERIZAÇÃO DOS SEGMENTOS:\n")
print("="*120)

for seg in sorted(df_cluster['Segmento'].unique()):
    seg_data = segment_profile.loc[seg]
    
    print(f"\n{'='*120}")
    print(f"SEGMENTO {seg}")
    print(f"{'='*120}")
    print(f"👥 Tamanho: {seg_data['Num_Clientes']:,.0f} clientes ({seg_data['Pct_Total']:.1f}% do total)")
    print(f"\n📊 Perfil Demográfico:")
    print(f"   • Idade Média: {seg_data['Idade_Média']:.0f} anos (± {seg_data['Idade_Desvio']:.1f})")
    print(f"   • Renda Média: R$ {seg_data['Renda_Média']:,.2f} (± R$ {seg_data['Renda_Desvio']:,.2f})")
    print(f"   • Cartões por Cliente: {seg_data['Cartões_Média']:.1f} em média")
    print(f"   • Conta Adicional: {seg_data['Pct_Conta_Adicional']*100:.1f}% possuem")
    print(f"   • Tempo Médio como Cliente: {seg_data['Tempo_Médio_Anos']:.1f} anos")
    
    # Analisar cidade/estado predominante
    seg_geo = df_with_segments[df_with_segments['Segmento'] == seg]
    top_estado = seg_geo['Estado'].mode()[0] if len(seg_geo) > 0 else 'N/A'
    top_cidade = seg_geo['Cidade'].mode()[0] if len(seg_geo) > 0 else 'N/A'
    print(f"\n🗺️ Localização:")
    print(f"   • Estado predominante: {top_estado}")
    print(f"   • Cidade predominante: {top_cidade}")

## 12. Comparação Visual entre Segmentos

In [ ]:
# Radar chart comparando segmentos
from math import pi

# Normalizar dados para radar chart (0-1)
radar_data = df_cluster.groupby('Segmento').agg({
    'Idade': 'mean',
    'Renda_Anual': 'mean',
    'Numero_Cartoes': 'mean',
    'Possui_Conta_Adicional_Bin': 'mean',
    'Tempo_Cliente_Anos': 'mean'
})

# Normalizar cada coluna (min-max scaling)
for col in radar_data.columns:
    min_val = radar_data[col].min()
    max_val = radar_data[col].max()
    radar_data[col] = (radar_data[col] - min_val) / (max_val - min_val) if max_val > min_val else 0

# Criar radar chart
fig = go.Figure()

categories = ['Idade', 'Renda', 'Num Cartões', 'Conta Adicional', 'Tempo Cliente']

for seg in radar_data.index:
    fig.add_trace(go.Scatterpolar(
        r=radar_data.loc[seg].values.tolist() + [radar_data.loc[seg].values[0]],
        theta=categories + [categories[0]],
        fill='toself',
        name=f'Segmento {seg}'
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True,
    title='🕸️ Radar Chart - Comparação de Perfis dos Segmentos (Normalizado)',
    height=600
)
fig.show()

In [ ]:
# Gráficos de barras agrupados comparando segmentos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Idade Média por Segmento', 'Renda Média por Segmento',
                    'Número Médio de Cartões', 'Tempo Médio como Cliente'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

segment_stats = df_cluster.groupby('Segmento').agg({
    'Idade': 'mean',
    'Renda_Anual': 'mean',
    'Numero_Cartoes': 'mean',
    'Tempo_Cliente_Anos': 'mean'
}).reset_index()

colors = px.colors.qualitative.Set2

# Idade
fig.add_trace(
    go.Bar(x=segment_stats['Segmento'], y=segment_stats['Idade'],
          marker_color=colors[0], name='Idade', showlegend=False),
    row=1, col=1
)

# Renda
fig.add_trace(
    go.Bar(x=segment_stats['Segmento'], y=segment_stats['Renda_Anual'],
          marker_color=colors[1], name='Renda', showlegend=False),
    row=1, col=2
)

# Cartões
fig.add_trace(
    go.Bar(x=segment_stats['Segmento'], y=segment_stats['Numero_Cartoes'],
          marker_color=colors[2], name='Cartões', showlegend=False),
    row=2, col=1
)

# Tempo
fig.add_trace(
    go.Bar(x=segment_stats['Segmento'], y=segment_stats['Tempo_Cliente_Anos'],
          marker_color=colors[3], name='Tempo', showlegend=False),
    row=2, col=2
)

fig.update_layout(height=700, title_text="📊 Comparação de Métricas entre Segmentos")
fig.show()

In [ ]:
# Box plots comparativos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição de Idade', 'Distribuição de Renda',
                    'Distribuição de Cartões', 'Distribuição Tempo Cliente')
)

for seg in sorted(df_cluster['Segmento'].unique()):
    seg_data = df_cluster[df_cluster['Segmento'] == seg]
    
    # Idade
    fig.add_trace(
        go.Box(y=seg_data['Idade'], name=f'Seg {seg}', showlegend=False),
        row=1, col=1
    )
    
    # Renda
    fig.add_trace(
        go.Box(y=seg_data['Renda_Anual'], name=f'Seg {seg}', showlegend=False),
        row=1, col=2
    )
    
    # Cartões
    fig.add_trace(
        go.Box(y=seg_data['Numero_Cartoes'], name=f'Seg {seg}', showlegend=False),
        row=2, col=1
    )
    
    # Tempo
    fig.add_trace(
        go.Box(y=seg_data['Tempo_Cliente_Anos'], name=f'Seg {seg}', showlegend=False),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="📦 Box Plots - Distribuições por Segmento")
fig.show()

## 13. Distribuição Geográfica por Segmento

In [ ]:
# Análise geográfica por segmento
geo_segment = df_with_segments.groupby(['Estado', 'Segmento']).size().reset_index(name='Count')

# Heatmap Estado x Segmento
pivot_geo = geo_segment.pivot(index='Estado', columns='Segmento', values='Count').fillna(0)

fig = go.Figure(data=go.Heatmap(
    z=pivot_geo.values,
    x=[f'Segmento {i}' for i in pivot_geo.columns],
    y=pivot_geo.index,
    colorscale='YlOrRd',
    text=pivot_geo.values.astype(int),
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Num Clientes")
))

fig.update_layout(
    title='🗺️ Distribuição Geográfica dos Segmentos por Estado',
    xaxis_title='Segmento',
    yaxis_title='Estado',
    height=500
)
fig.show()

In [ ]:
# Gráfico de barras empilhadas - Estados por Segmento
fig = go.Figure()

for seg in sorted(geo_segment['Segmento'].unique()):
    seg_data = geo_segment[geo_segment['Segmento'] == seg]
    fig.add_trace(go.Bar(
        name=f'Segmento {seg}',
        x=seg_data['Estado'],
        y=seg_data['Count']
    ))

fig.update_layout(
    barmode='stack',
    title='📊 Composição de Segmentos por Estado (Empilhado)',
    xaxis_title='Estado',
    yaxis_title='Número de Clientes',
    height=500
)
fig.show()

## 14. Insights e Recomendações Estratégicas

In [ ]:
# Sumário executivo dos segmentos
print("\n" + "="*120)
print("🎯 SUMÁRIO EXECUTIVO - SEGMENTAÇÃO DE CLIENTES PRICELESS BANK")
print("="*120)

print(f"\n📊 VISÃO GERAL:")
print(f"   • Total de Clientes Analisados: {len(df_cluster):,}")
print(f"   • Número de Segmentos Identificados: {n_clusters}")
print(f"   • Taxa de Dados Completos: {(len(df_cluster)/len(df)*100):.1f}%")

print(f"\n💼 PRINCIPAIS ACHADOS:")

# Segmento com maior renda
seg_maior_renda = segment_profile['Renda_Média'].idxmax()
print(f"   • Segmento de Maior Renda: Segmento {seg_maior_renda} (R$ {segment_profile.loc[seg_maior_renda, 'Renda_Média']:,.2f})")

# Segmento com mais clientes
seg_maior_base = segment_profile['Num_Clientes'].idxmax()
print(f"   • Segmento com Maior Base: Segmento {seg_maior_base} ({segment_profile.loc[seg_maior_base, 'Num_Clientes']:,.0f} clientes)")

# Segmento mais jovem
seg_mais_jovem = segment_profile['Idade_Média'].idxmin()
print(f"   • Segmento Mais Jovem: Segmento {seg_mais_jovem} ({segment_profile.loc[seg_mais_jovem, 'Idade_Média']:.0f} anos)")

# Segmento com mais cartões
seg_mais_cartoes = segment_profile['Cartões_Média'].idxmax()
print(f"   • Segmento com Mais Cartões: Segmento {seg_mais_cartoes} ({segment_profile.loc[seg_mais_cartoes, 'Cartões_Média']:.1f} cartões/cliente)")

print(f"\n🎯 OPORTUNIDADES IDENTIFICADAS:")
print(f"   1. Cross-sell de cartões para segmentos com baixa penetração")
print(f"   2. Programas de retenção para segmentos com menor tempo médio")
print(f"   3. Produtos premium para segmentos de alta renda")
print(f"   4. Ofertas regionalizadas baseadas na concentração geográfica")
print(f"   5. Estratégias digitais para segmentos mais jovens")

print("\n" + "="*120)

## 15. Exportar Resultados

In [ ]:
# Salvar dados com segmentos
output_file = 'clientes_segmentados.csv'
df_with_segments.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✓ Arquivo exportado: {output_file}")

# Salvar perfil dos segmentos
profile_file = 'perfil_segmentos.csv'
segment_profile.to_csv(profile_file, encoding='utf-8-sig')
print(f"✓ Perfil dos segmentos exportado: {profile_file}")

print("\n✅ Análise completa! Todos os segmentos foram identificados e visualizados.")